In [ ]:
#Code for FL_Zone_9a_9b_Scrub_Incursion_Restoratoin Project 
#Preprocessing of Rasters for Random Forest
#GEOG561
#Mike Karasoff

import arcpy
import numpy as np
import time

DBPATH=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb"
DATA_PATH=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\Data"

arcpy.env.workspace=DBPATH


demFile = r"%s\flidar_mosaic_m\flidar_mosaic_m.gdb\flidar_mosaic_m" % DATA_PATH
spatialRef = arcpy.Describe(demFile).spatialReference
defaultGridSz = 25


In [ ]:
#Create Extent Feature.  This is the overlay of the DRASTIC and Subsurface Geological Data
waterTbl = r"%s\DRASTIC_for_the_Surficial_Aquifer\DRASTIC_for_the_Surficial_Aquifer.shp" % DATA_PATH
subSurfaceGeog =  r"%s\FL_Soil_Geology\FGS_STATEMAP_GEOLOGY.gdb\FGS_STATEMAP_GEOLOGY" % DATA_PATH

#This will make sure we aren't selecting data from the panhandle
initMaskExtent = "404732.2623999994 334500.8658999996 697135.871199999 733658.6742000002"

#Get the intersection of the two
with arcpy.EnvManager(outputCoordinateSystem=spatialRef, 
                                              parallelProcessingFactor="75%", 
                                              extent=initMaskExtent):
    arcpy.analysis.Intersect([subSurfaceGeog, 'zone9'], 'zoneGeog_isect', 'ONLY_FID')

#Note: The extent gets reduced further...
with arcpy.EnvManager(outputCoordinateSystem=spatialRef, 
                                              parallelProcessingFactor="75%", 
                                              extent="MINOF"):
    arcpy.analysis.Intersect([waterTbl, 'zoneGeog_isect'], 'rfMaskFeature_isect', 'ONLY_FID') 

#Dissolve to remove boundaries for extent
arcpy.management.Dissolve('rfMaskFeature_isect', 'rfMaskFeature')
#Clean up
arcpy.management.Delete(r"'rfMaskFeature_isect'; 'zoneGeog_isect'")
arcpy.management.Delete(r"'rfMaskFeature_isect'; 'zoneGeog_isect'")



In [ ]:
#Get extent for future stuff
desc = arcpy.Describe('rfMaskFeature')
xmin = desc.extent.XMin
xmax = desc.extent.XMax
ymin = desc.extent.YMin
ymax = desc.extent.YMax
rfMaskExtent="%s %s %s %s" % (xmin, ymin, xmax, ymax)

In [ ]:
#Create a snap raster/mask raster from the extent feature
with arcpy.EnvManager(outputCoordinateSystem=spatialRef,
                      pyramid="NONE", 
                      cellSize=defaultGridSz):
    arcpy.conversion.FeatureToRaster('rfMaskFeature', "OBJECTID", "rfMaskRaster", defaultGridSz)


In [ ]:
def fillRaster(inRaster):
    
    prop = arcpy.GetRasterProperties_management(inRaster, "MINIMUM")
    minVal = int(prop.getOutput(0))
    prop = arcpy.GetRasterProperties_management(inRaster, "MAXIMUM")
    maxVal = int(prop.getOutput(0))
    prop = arcpy.GetRasterProperties_management(inRaster, "VALUETYPE") 
    valType = int(prop.getOutput(0))

    valueRange = {  0 : (0,1),
                    1 : (0, 3),
                    2 : (0, 15),
                    3 : (0, 255),
                    4 : (-128, 127),
                    5 : (0, 65535),
                    6 : (-32768, 32767),
                    7 : (0, 42949672957),
                    8 : (-2147483648, 21474836477),
                    9 : (-3.402823466e+38 , 3.402823466e+38),
                    10 : (0, 18446744073709551616)
                 }
    try:
        minMax = valueRange[valType]
    except:
        print("This value %s is likely complex. Consider another way." % valType )


    if minVal > minMax[0]:
        nullValue = minMax[0]
    elif maxVal < minMax[1]:
        nullValue = minMax[1]
    else:
        nullValue = 0

    print("Setting null values in raster %s to: %s" % (inRaster, nullValue))    
    
    with arcpy.EnvManager(outputCoordinateSystem=spatialRef, extent=rfMaskExtent):
        filledRaster = arcpy.sa.Con(arcpy.sa.IsNull(inRaster), nullValue, inRaster, "Value = 1")  

    return filledRaster

In [ ]:
#Copy Raster Function.  This will copy a raster to the database with the defined paramters for the rf
def rfCopyMaskedRaster(inRaster, outName):
    filledRaster = fillRaster(inRaster)

    with arcpy.EnvManager(outputCoordinateSystem=spatialRef, 
                      snapRaster='rfMaskRaster', 
                      cellSize='rfMaskRaster', 
                      mask="rfMaskRaster"):
        
        maskedRaster = arcpy.sa.ExtractByMask(filledRaster, "rfMaskRaster");
        maskedRaster.save(outName)        

In [ ]:
def update_subsurface_epoch():
    infile=r"%s\FL_Soil_Geology\FGS_STATEMAP_GEOLOGY.gdb\FGS_STATEMAP_GEOLOGY" % DATA_PATH
    
    arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")
    
    time.sleep(2)
    
    field="GEOLOGIC_HISTORY"
    
    with arcpy.da.UpdateCursor(infile, (field,)) as cursor:
        for row in cursor:
            try:
                row[0]=row[0].replace(" ", "")
                row[0]=row[0].upper()
            except:
                row[0]=""

            cursor.updateRow(row)
    
    arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")


In [ ]:
#This expands the lithology field in the FGS_STATEMAP_GEOLOGY feature and updates the table so 
# attributes can be mapped accurately
#The original file only contains this information as a comma delimited list in an attribute field,
# in natural languge, with mixed order, case, and plurality (e.g. "shells, organic" in one attribute
# and then "organics, Shell" in another). This fixes the language for uniformity
# and updates the table with binary indicators for the varous lithologites.

def update_lithology_attributes():
    infile=r"%s\FL_Soil_Geology\FGS_STATEMAP_GEOLOGY.gdb\FGS_STATEMAP_GEOLOGY" % DATA_PATH
    arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")

    geogTbl = arcpy.da.TableToNumPyArray (infile, 'Lithology', skip_nulls=True)
    lithos = set()

    #Find the various lithologies and fix 
    for lithology in geogTbl["Lithology"]:
        try: 
            lithology = lithology.lower()
            lithology = lithology.replace(" ", "")
            lithology = lithology.replace("s,", ",")
            if lithology[-1] == 's': lithology = lithology[:-1]
        except:
            continue
        
        lithos = lithos | set(lithology.split(','))
        
    arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")

    #should be {'organic', 'sand', 'phosphate', 'dolostone', 'shell', 'heavymineral', 'clay', 'limestone'}
    print("Adding Fields:",  lithos)

    for litho in lithos:
        arcpy.AddField_management(infile, litho, "SHORT")
        print("Adding Field %s" % litho)

    fields=['Lithology']
    fields.extend(list(lithos))

    fieldMap = {fields[x] : x for x in range(len(fields))}
    print("Mapping field indexes:", fieldMap)
    
    with arcpy.da.UpdateCursor(infile, fields) as cursor:
        for row in cursor:
            print("Row in", row)
            try: 
                row[1:] = [0] * (len(row) - 1)

                lithology = row[fieldMap['Lithology']] 
                lithology = lithology.lower()
                lithology = lithology.replace(" ", "")
                lithology = lithology.replace("s,", ",")
                if lithology[-1] == 's': lithology = lithology[:-1]

            except:
                cursor.updateRow(row)
                continue


            for soil in lithology.split(','):
                row[fieldMap[soil]] = 1

            print("Row out", row)
            #Update the cursor with the updated list
            cursor.updateRow(row)

    arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")


In [ ]:
#Copy Feature Function.  This will copy a reature to the database with the defined paramters for the rf
def rfCreateMaskedRaster(inFeature, rasterField, outName):
    tempFeature = "%s_temp" % outName
    arcpy.management.Delete(tempFeature)

    print(inFeature)
    print(tempFeature)

    #ArcGIS is acting weird, claiming features aren't 
    arcpy.management.SelectLayerByAttribute(inFeature, "CLEAR_SELECTION")

    arcpy.analysis.Intersect([inFeature, 'rfMaskFeature'], tempFeature) 
    
    with arcpy.EnvManager(outputCoordinateSystem=spatialRef, 
                      snapRaster="rfMaskRaster", 
                      pyramid="NONE", 
                      extent="DEFAULT"):
        
        
        arcpy.conversion.FeatureToRaster(tempFeature, rasterField, outName, defaultGridSz)
    
    arcpy.management.Delete(tempFeature)


In [ ]:
#Create Feature with Psudo-Absence Data for Scrub

infile=r"%s\FL_Classified_Land_Cover.gdb\CLC_v3_4_poly" % DATA_PATH
arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")

polys = arcpy.da.TableToNumPyArray (infile, 'NAME_SITE', skip_nulls=True)



listOfSites = list(np.unique(polys['NAME_SITE']))

#Generate List of Absense Sites
absenceSites = set()
exludedTerms = ["Marsh", "Swamp", "Marine", "Floodplain", "Wetland", "Surf", "Cutthroat"]
for exludedTerm in exludedTerms:
    absenceSites = absenceSites | set([s for s in listOfSites if exludedTerm in s])

print("Absense Sites:", absenceSites)    
    
#Build SQL Query
ppaSelQueary=""
for absenceSite in absenceSites:
    ppaSelQueary = ppaSelQueary + str("NAME_SITE = \'%s\' Or " % absenceSite)

#Add Presense Data to Query    
ppaSelQueary = ppaSelQueary + "NAME_SITE = \'Scrub\'"

#Select by Attribute
ppaScrub = arcpy.management.SelectLayerByAttribute(infile, 
                                    where_clause = ppaSelQueary)


arcpy.analysis.Intersect([ppaScrub, 'zone9'], 'ppaScrubZone9')
arcpy.management.Delete(ppaScrub)


with arcpy.EnvManager(outputCoordinateSystem=spatialRef, extent=rfMaskExtent):
     arcpy.gapro.OverlayLayers('ppaScrubZone9', 
                               'rfMaskFeature', 
                               'ppa_scrub_training_features', 
                               overlay_type = 'INTERSECT')


####Cleanup
arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")

arcpy.AddField_management(infile, "Presence", "SHORT")

with arcpy.da.UpdateCursor(infile, ["NAME_SITE", "Presence"]) as cursor:
    for row in cursor:
        if row[0] == "Scrub":
            row[1] = 1;
        else:
            row[1] = 0;
        
        cursor.updateRow(row)


In [ ]:
print("Status for training data")

arcpy.management.SelectLayerByAttribute('ppa_scrub_training_features', "CLEAR_SELECTION")
arcpy.management.SelectLayerByAttribute('ppaScrubZone9', "CLEAR_SELECTION")

areas = arcpy.da.TableToNumPyArray ('ppaScrubZone9', 'SHAPE_Area', skip_nulls=True)
originalDataArea = areas["SHAPE_Area"].sum()
print("Original Area %s m^2"  % originalDataArea)

areas = arcpy.da.TableToNumPyArray ('ppa_scrub_training_features', 'SHAPE_Area', skip_nulls=True)
maskedDataArea = areas["SHAPE_Area"].sum()
print("Area after Masking %s m^2"  % maskedDataArea)

knownScrubZone9 = arcpy.management.SelectLayerByAttribute('ppa_scrub_training_features', 
                                                            where_clause = "NAME_SITE = \'Scrub\'")

areas = arcpy.da.TableToNumPyArray (knownScrubZone9, 'SHAPE_Area', skip_nulls=True)
presenceDataArea = areas["SHAPE_Area"].sum()
print("Area of presence data in  %s m^2"  % presenceDataArea)

knownScrubZone9Area = areas["SHAPE_Area"].sum()
print("Original Area %s m^2"  % originalDataArea)

maskedPct = 100 * (maskedDataArea/originalDataArea)
print("Percentage of original area after masking %s%%" % maskedPct)

paRatio = 100 * (presenceDataArea/maskedDataArea)
print("Percentage of presence data in study area %s%%" % paRatio)

arcpy.management.SelectLayerByAttribute('ppa_scrub_training_features', "CLEAR_SELECTION")
arcpy.management.SelectLayerByAttribute('ppaScrubZone9', "CLEAR_SELECTION")


In [ ]:
#Create Clipped Rasters from Raster
flidar_mosaic_m = r"%s\flidar_mosaic_m\flidar_mosaic_m.gdb\flidar_mosaic_m" % DATA_PATH 

rf_dem = arcpy.sa.ExtractByMask(flidar_mosaic_m, "rfMaskRaster")
rf_dem.save('rf_dem')        
    

In [ ]:
arcpy.management.CopyRaster(rf_dem, "rf_dem_32", pixel_type="32_BIT_SIGNED")


In [ ]:
scaled_100x_dem = arcpy.Raster("rf_dem_32") * 100
with arcpy.EnvManager(outputCoordinateSystem="PROJCS['Albers Conical Equal Area [Florida Geographic Data Library]',GEOGCS['GCS_North_American_1983_HARN',DATUM['D_North_American_1983_HARN',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',400000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-84.0],PARAMETER['Standard_Parallel_1',24.0],PARAMETER['Standard_Parallel_2',31.5],PARAMETER['Central_Parallel',24.0],UNIT['Meter',1.0]],VERTCS['NAVD_1988',VDATUM['North_American_Vertical_Datum_1988'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]]"):
    scaled_100x_dem_slope = arcpy.sa.Slope(scaled_100x_dem, "DEGREE", 1, "PLANAR", "METER") 
    scaled_100x_dem_slope_slope = arcpy.sa.Slope(scaled_100x_dem_slope, "DEGREE", 1, "PLANAR", "METER") 
    dem_slope = arcpy.sa.Slope('rf_dem', "DEGREE", 1, "PLANAR", "METER") 

scaled_100x_dem_slope.save('rf_scaled_100x_dem_slope')
scaled_100x_dem_slope_slope.save('rf_scaled_100x_dem_slope_slope')
dem_slope.save('rf_dem_slope')


In [ ]:
scaled_100x_dem.save('scaled_100x_dem')

In [ ]:
out_raster = arcpy.sa.Slope("rf_dem_slope", "DEGREE", 1, "PLANAR", "METER"); out_raster.save(r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\rf_dem_slope_slope")       

In [ ]:

with arcpy.EnvManager(outputCoordinateSystem="PROJCS['Albers Conical Equal Area [Florida Geographic Data Library]',GEOGCS['GCS_North_American_1983_HARN',DATUM['D_North_American_1983_HARN',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',400000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-84.0],PARAMETER['Standard_Parallel_1',24.0],PARAMETER['Standard_Parallel_2',31.5],PARAMETER['Central_Parallel',24.0],UNIT['Meter',1.0]],VERTCS['NAVD_1988',VDATUM['North_American_Vertical_Datum_1988'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]]", cellSize=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\rf_dem"):
    arcpy.ddd.Slope("rf_dem", r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\Slope_rf_dem2", "PERCENT_RISE", 1, "GEODESIC", "METER")

In [ ]:
out_raster = arcpy.sa.ExtractByMask(r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\Data\flidar_mosaic_m\flidar_mosaic_m.gdb\flidar_mosaic_m", "rfMaskFeature"); 
out_raster.save(r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\rf_dem")

In [ ]:
out_raster.save(r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\rf_dem")

In [ ]:
#Create Slope from Elevation.  This will take a few minutes.
out_raster = arcpy.sa.Slope(rf_dem, method = "GEODESIC")
out_raster.save('rf_slope')


In [ ]:
#update_lithology_attributes()
#update_subsurface_epoch()

In [ ]:
#Create Rasters from Features
geog_infile=r"%s\FL_Soil_Geology\FGS_STATEMAP_GEOLOGY.gdb\FGS_STATEMAP_GEOLOGY" % DATA_PATH

geog_fields = ['GEOLOGIC_HISTORY', 
                   'organic', 
                   'sand', 
                   'phosphate', 
                   'dolostone', 
                   'shell', 
                   'heavymineral', 
                   'clay', 
                   'limestone']

for geog_field in geog_fields:
    rasterName="rf_subsurface_geog_%s" % geog_field
    rfCreateMaskedRaster(geog_infile, geog_field, rasterName)

features=[]
features.append(("%s/USDA_NRCS_Soils/nrcs_soils_sep18.shp" % DATA_PATH, 
                "DRAINAGECL", 
                "rf_drainage"))

features.append(("%s/USDA_NRCS_Soils/nrcs_soils_sep18.shp" % DATA_PATH, 
                 "MUKEY", 
                 "rf_top_soil_detail"))


features.append((r"%s\USDA_FL_Soil\spatial\gsmsoilmu_a_fl.shp" % DATA_PATH, 
                "MUKEY", 
                "rf_top_soil"))

features.append((r"%s\DRASTIC_for_the_Surficial_Aquifer\DRASTIC_for_the_Surficial_Aquifer.shp" % DATA_PATH, 
                 "DW", 
                 "rf_water_table_depth"))


features.append(("%s/USDA_NRCS_Soils/nrcs_soils_sep18.shp" % DATA_PATH, 
                "FLODFREQMA", 
                "rf_flooding"))

features.append((r"%s\DRASTIC_for_the_Surficial_Aquifer\DRASTIC_for_the_Surficial_Aquifer.shp" % DATA_PATH, 
                 "HC", 
                 "rf_hydraulic_conductivity"))

for feature in features:
    rfCreateMaskedRaster(feature[0], feature[1], feature[2])



    

In [ ]:
features=[]
features.append(("%s/USDA_NRCS_Soils/nrcs_soils_sep18.shp" % DATA_PATH, 
                "TEXTURE", 
                "rf_texture"))

features.append(("%s/USDA_NRCS_Soils/nrcs_soils_sep18.shp" % DATA_PATH, 
                "HYDRICRATI", 
                "rf_hydric"))
for feature in features:
    rfCreateMaskedRaster(feature[0], feature[1], feature[2])

In [ ]:
features=[]
features.append(("%s/USDA_NRCS_Soils/nrcs_soils_sep18.shp" % DATA_PATH, 
                "HYDRGRP", 
                "rf_hdg"))
for feature in features:
    rfCreateMaskedRaster(feature[0], feature[1], feature[2])

In [ ]:
geog_infile=r"%s\FL_Soil_Geology\FGS_STATEMAP_GEOLOGY.gdb\FGS_STATEMAP_GEOLOGY" % DATA_PATH

geog_fields = ['GEOLOGIC_HISTORY', 
                   'organic', 
                   'sand', 
                   'phosphate', 
                   'dolostone', 
                   'shell', 
                   'heavymineral', 
                   'clay', 
                   'limestone']

    
for geog_field in geog_fields:
    rasterName="rf_subsurface_geog_%s" % geog_field
    rfCreateMaskedRaster(geog_infile, geog_field, rasterName)



In [ ]:
infile = r"%s\DRASTIC_for_the_Surficial_Aquifer\DRASTIC_for_the_Surficial_Aquifer.shp" % DATA_PATH

for field in arcpy.ListFields(infile):
    print(field.name)

In [ ]:
#Create water table depth raster
infile=r"%s\DRASTIC_for_the_Surficial_Aquifer\DRASTIC_for_the_Surficial_Aquifer.shp" % DATA_PATH
arcpy.management.SelectLayerByAttribute(infile, "CLEAR_SELECTION")

arcpy.conversion.FeatureToRaster(infile, "DW", "water_table_depth", 15)

In [ ]:
arcpy.AddField_management("ppa_scrub_training_features", "Presence", "SHORT")

with arcpy.da.UpdateCursor("ppa_scrub_training_features", ["NAME_SITE", "Presence"]) as cursor:
    for row in cursor:
        if row[0] == "Scrub":
            row[1] = 1;
        else:
            row[1] = 0;
        
        cursor.updateRow(row)


In [ ]:
subsurf_names = [  'rf_subsurface_geog_organic', 
                   'rf_subsurface_geog_sand', 
                   'rf_subsurface_geog_phosphate', 
                   'rf_subsurface_geog_dolostone', 
                   'rf_subsurface_geog_shell', 
                   'rf_subsurface_geog_heavymineral', 
                   'rf_subsurface_geog_clay', 
                   'rf_subsurface_geog_limestone']


rasters=[]


for subsurf_name in subsurf_names:
    rasters.append(arcpy.Raster(subsurf_name))

iterator = 1

rf_subsurface_geog_comp = rasters[0]

for raster in rasters[1:]:
    rf_subsurface_geog_comp = rf_subsurface_geog_comp + (raster * (2**iterator))
    iterator = iterator+1

rf_subsurface_geog_comp

In [ ]:
rf_subsurface_geog_comp.save("rf_subsurface_geog_comp")

In [ ]:
#Get CS observations from known zone9 scrub lands from CLC
arcpy.management.AddField("csScrubCommunityPa", "Presence", "SHORT", )
arcpy.management.CalculateField("csScrubCommunityPa", "Presence", "0", "PYTHON3", '', "TEXT")
arcpy.analysis.Intersect(['csScrubCommunityPa', 'rfMaskFeature'], 'pa_scrub_community') 



In [ ]:
#Get CS observations from known zone9 scrub lands from CLC
arcpy.management.AddField("csScrubPlantsZone9Communities", "Presence", "SHORT", )
arcpy.management.CalculateField("csScrubPlantsZone9Communities", "Presence", "1", "PYTHON3", '', "TEXT")
arcpy.analysis.Intersect(['csScrubPlantsZone9Communities', 'rfMaskFeature'], 'p_scrub_community') 


In [ ]:
arcpy.management.Merge(["p_scrub_community", "pa_scrub_community"], "ppa_cs_scrub_community")

In [ ]:
with arcpy.EnvManager(outputCoordinateSystem="PROJCS['Albers Conical Equal Area [Florida Geographic Data Library]',GEOGCS['GCS_North_American_1983_HARN',DATUM['D_North_American_1983_HARN',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',400000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-84.0],PARAMETER['Standard_Parallel_1',24.0],PARAMETER['Standard_Parallel_2',31.5],PARAMETER['Central_Parallel',24.0],UNIT['Meter',1.0]],VERTCS['NAVD_1988',VDATUM['North_American_Vertical_Datum_1988'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]]", parallelProcessingFactor="90%", mask=None):
    arcpy.stats.Forest("TRAIN", r"Training Data\ppa_cs_scrub_community_training", "Presence", "CATEGORICAL", None, None, r"'Explanatory Variables\rf_drainage' true;'Explanatory Variables\rf_flooding' true;'Explanatory Variables\rf_hdg' true;'Explanatory Variables\rf_hydraulic_conductivity' true;'Explanatory Variables\rf_hydric' true;'Explanatory Variables\rf_subsurface_geog_clay' true;'Explanatory Variables\rf_subsurface_geog_comp' true;'Explanatory Variables\rf_subsurface_geog_GEOLOGIC_HISTORY' true;'Explanatory Variables\rf_texture' true;'Explanatory Variables\rf_water_table_depth' false;expdata\softened_scaled_100x_dem #;expdata\Curvature_softened_scaled_100x_dem #;expdata\Slope_softened_scaled_100x_dem #;gradient_e_softened_scale.tif #", None, None, None, None, None, None, r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\o_features", r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\o_importance", "FALSE", 1000, None, None, 100, 5, 50, r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\o_confusion", r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\o_validation", "TRUE", 2, "FALSE")